In [ ]:
"""  
reference to codes at  
https://code.earthengine.google.com/?accept_repo=users%2Fujavalgandhi%2FEnd-to-End-Projects



Note:      

the following is the javascript code used in google earth engine for downloading selected locations and time intervals.

calculating annual means inside google earth engine will reduce file sizes and make downloading process easier.   

"""


Var roi: Table projects/ee-letustry/assets/location1    # selecting ROI (Region of interest) and uploading shapefile.       

var modis = ee.ImageCollection("MODIS/006/MOD13Q1");	# Modis dataset of NDVI

var filtered = modis
  .filter(ee.Filter.date('2002-01-01', '2021-12-31'))     # selecting time interval


var modisNDVI = filtered.select('NDVI')
var scaledNDVI = modisNDVI.map(function(image){
  return image.multiply(0.0001)
  .copyProperties(image,['system:time_start','system:time_end']);
});


var triplets = scaledNDVI.map(function(image) {
  var withStats = image.reduceRegions({
  collection: roi,
  reducer: ee.Reducer.mean().setOutputs(['ndvi']),
  scale: 250
  }).map(function(feature) {
    return feature.set('imageId', image.id())
  })
  return withStats
}).flatten()

var format = function(table, rowId, colId) {
  var rows = table.distinct(rowId); 
  var joined = ee.Join.saveAll('matches').apply({
    primary: rows, 
    secondary: table, 
    condition: ee.Filter.equals({
      leftField: rowId, 
      rightField: rowId
    })
  });
         
  return joined.map(function(row) {
      var values = ee.List(row.get('matches'))
        .map(function(feature) {
          feature = ee.Feature(feature);
          var ndvi = ee.List([feature.get('ndvi'), -9999])
            .reduce(ee.Reducer.firstNonNull())
          return [feature.get(colId), ee.Number(ndvi).format('%.2f')];
        });
      return row.select([rowId]).set(ee.Dictionary(values.flatten()));
    });
};
// loc
var timeSeriesResults = format(triplets, 'loc', 'imageId');
print(timeSeriesResults.first()) 


Export.table.toDrive({
  collection: timeSeriesResults,
  description: 'MODIS_NDVI_Series',
  folder: 'earthengine',
  fileNamePrefix: 'modis_ndvi_series',
  fileFormat: 'CSV'})






